# Dependencies

In [23]:
pacman::p_load(dplyr)

# Functions

In [24]:
source("/Users/anabbi/git/ped_CapTCRseq/R/ggplot2_theme.R")
source("/Users/anabbi/git/ped_CapTCRseq/R/color_schemes.R")

# Paths

In [25]:
datapath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Data/"
plotpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Plots/"
manifestpath <- "/Users/anabbi/OneDrive - UHN/Documents/INTERCEPT/Manifests/"
gitpath <- "/Users/anabbi/git/ped_CapTCRseq/"

In [26]:
h4hpath <- "/Users/anabbi/Desktop/H4H/INTERCEPT/"

# Main

bind all metadata

In [27]:
meta_div <- readr::read_rds(file = paste0(datapath,"capTCRseq/meta_div_goodsamples.rds"))

In [28]:
allAdaptive <- readr::read_rds(file = paste0(datapath, "Adaptivedatasets/allAdaptive_good.rds"))

In [29]:
# remove those with no TCR data
meta_div <- meta_div[ !is.na(meta_div$file),]

In [30]:
meta_div$study <- "INTERCEPT"
meta_div$Agegroup <- "Child"
meta_div$Sex <- NA
meta_div$disease <- meta_div$cancergroup
meta_div$sample_name <- meta_div$basename

In [31]:
allAdaptive$sampletype <- "PBMC"
allAdaptive$sampletype[grepl("Cord", allAdaptive$sample_name )] <- "Cord blood"

In [32]:
head(allAdaptive)

,sample_name,study,Agegroup,Sex,disease,subject,age,sampletype
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
21,Peds_Ctrl_2,Henderson,Child,NA,Peds healthy Control,Peds_Ctrl_2,NA,PBMC
22,MAS_3,Henderson,Child,NA,MAS,MAS_3,NA,PBMC
23,Peds_Febrile_Ctrl_3,Henderson,Child,NA,Peds febrile Control,Peds_Febrile_Ctrl_3,NA,PBMC
25,KD_4,Henderson,Child,NA,KD,KD_4,NA,PBMC
26,KD_3,Henderson,Child,NA,KD,KD_3,NA,PBMC
29,sJIA_7,Henderson,Child,NA,sJIA,sJIA_7,NA,PBMC


In [33]:
# replace age with Age in allAdative
colnames(allAdaptive)[colnames(allAdaptive) == "age"] <- "Age"
# subject in allAdaptive is the same as Patient in meta_div
colnames(allAdaptive)[colnames(allAdaptive) == "subject"] <- "Patient"

In [34]:
mycols <- intersect(colnames(meta_div), colnames(allAdaptive))

In [35]:
mycols

[1] "Patient"     "Age"         "sampletype"  "study"       "Agegroup"   
[6] "Sex"         "disease"     "sample_name"

In [36]:
all_metadata <- rbind(allAdaptive[,mycols], meta_div[,mycols])

In [37]:
dim(all_metadata)

[1] 1677    8

In [38]:
head(all_metadata)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
21,Peds_Ctrl_2,NA,PBMC,Henderson,Child,NA,Peds healthy Control,Peds_Ctrl_2
22,MAS_3,NA,PBMC,Henderson,Child,NA,MAS,MAS_3
23,Peds_Febrile_Ctrl_3,NA,PBMC,Henderson,Child,NA,Peds febrile Control,Peds_Febrile_Ctrl_3
25,KD_4,NA,PBMC,Henderson,Child,NA,KD,KD_4
26,KD_3,NA,PBMC,Henderson,Child,NA,KD,KD_3
29,sJIA_7,NA,PBMC,Henderson,Child,NA,sJIA,sJIA_7


In [39]:
table(all_metadata$study)


        Emerson       Henderson       INTERCEPT        Mitchell Mitchell_Denver 
            773              25             545             203             131 

In [40]:
#save all_metadata
readr::write_rds(all_metadata, file = paste0(datapath, "all_metadata_good.rds"))

## All PBMC data

In [41]:
# remove cfDNA samples
all_metadata_pbmc <- all_metadata[!grepl("cfDNA", all_metadata$sample_name),]

all_metadata_pbmc <- all_metadata_pbmc[!grepl("FFPE", all_metadata_pbmc$sample_name),]

In [42]:
dim(all_metadata_pbmc)

[1] 1390    8

In [43]:
table(all_metadata_pbmc$study)


        Emerson       Henderson       INTERCEPT        Mitchell Mitchell_Denver 
            773              25             258             203             131 

## Discovery set

In [44]:
discovery_metadata <- all_metadata_pbmc[ all_metadata_pbmc$study != "Mitchell_Denver",]

In [45]:
discovery_metadata <- discovery_metadata[ !grepl("Keck", discovery_metadata$sample_name),] # emerson validation set

In [46]:
#remove carey Cord samples
discovery_metadata <- discovery_metadata[ !grepl("Cord", discovery_metadata$sample_name),]

In [47]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 258   8

In [48]:
dim(discovery_metadata)

[1] 1140    8

#### sample intercept study

In [49]:
# subset to INTERCEPT study
tmp <- discovery_metadata[ discovery_metadata$study == "INTERCEPT",]

In [50]:
length(table(tmp$Patient)) # number of patients

[1] 112

In [51]:
myfreq <- as.data.frame(table(tmp$Patient, tmp$disease), stringsAsFactors = F)

In [52]:
#remove zero
myfreq <- myfreq[myfreq$Freq > 0,]

In [53]:
head(myfreq)

,Var1,Var2,Freq
,<chr>,<chr>,<int>
103,LFS_TCR_4535,LFS,1
104,LFS_TCR_4570,LFS,1
105,LFS_TCR_4602,LFS,1
106,LFS_TCR_4758,LFS,1
107,LFS_TCR_4759,LFS,1
108,LFS_TCR_4783,LFS,1


In [54]:
# randomly select 50% of patients from each disease group regardless of how many samples each patient has
set.seed(7)
myfreq %>% group_by(Var2) %>% sample_frac(0.5) -> discovery_sample_freq

In [55]:
discovery_sample_freq

Var1,Var2,Freq
<chr>,<chr>,<int>
LFS_TCR_5067,LFS,1
LFS_TCR_4602,LFS,1
LFS_TCR_4837,LFS,1
LFS_TCR_4758,LFS,1
LFS_TCR_4570,LFS,1
CHP_354,Leukemia,1
CHP_377,Leukemia,4
CHP_340,Leukemia,3
CHP_328,Leukemia,2


In [56]:
# all patients in discovery set
int_discovery_patients <- discovery_sample_freq$Var1

In [57]:
length(int_discovery_patients)

[1] 57

In [58]:
dim(discovery_metadata)

[1] 1140    8

In [59]:
dim(discovery_metadata[ discovery_metadata$study == "INTERCEPT",])

[1] 258   8

In [60]:
all_discovery_patients <- c(discovery_metadata$Patient[ discovery_metadata$study != "INTERCEPT"], int_discovery_patients)

In [61]:
discovery_metadata <- discovery_metadata[ discovery_metadata$Patient %in% all_discovery_patients,]

In [62]:
length(table(discovery_metadata$Patient[ discovery_metadata$study == "INTERCEPT"] ))

[1] 57

In [63]:
# CHP_407 is dupliucate sample one from lymphoma one from ffpe
dim(discovery_metadata)

[1] 1009    8

In [64]:
table(discovery_metadata$study)


  Emerson Henderson INTERCEPT  Mitchell 
      654        25       127       203 

## Validation set

In [65]:
head(all_metadata_pbmc)

,Patient,Age,sampletype,study,Agegroup,Sex,disease,sample_name
,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
21,Peds_Ctrl_2,NA,PBMC,Henderson,Child,NA,Peds healthy Control,Peds_Ctrl_2
22,MAS_3,NA,PBMC,Henderson,Child,NA,MAS,MAS_3
23,Peds_Febrile_Ctrl_3,NA,PBMC,Henderson,Child,NA,Peds febrile Control,Peds_Febrile_Ctrl_3
25,KD_4,NA,PBMC,Henderson,Child,NA,KD,KD_4
26,KD_3,NA,PBMC,Henderson,Child,NA,KD,KD_3
29,sJIA_7,NA,PBMC,Henderson,Child,NA,sJIA,sJIA_7


In [66]:
validation_metadata <- all_metadata_pbmc[ !all_metadata_pbmc$Patient %in% all_discovery_patients,]

In [67]:
dim(validation_metadata)

[1] 381   8

In [68]:
dim(discovery_metadata)

[1] 1009    8

In [69]:
# save discovery and validation metadata
readr::write_rds(discovery_metadata, file = paste0(datapath, "discovery_metadata.rds"))
readr::write_rds(validation_metadata, file = paste0(datapath, "validation_metadata.rds"))